In [1]:
import scipy
import pandas as pd
import numpy as np
import math
#import pymongo
#import random
import csv
import time
import re
import matplotlib.pyplot as plt; import matplotlib.pylab as pylab
#%matplotlib inline
pd.options.display.mpl_style = 'default'
pylab.rcParams['figure.figsize'] = 12, 6
from dateutil import parser
#import Quandl
#from pymongo import MongoClient
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import urllib

## Get A BeautifulSoup Object

In [2]:
class BloombergSearch:
    def __init__(self, search_term):
        self.search_term = search_term
        self.url_page1 = ('http://www.bloomberg.com/search?query=' + str(self.search_term))

    def get_search_soup(self):
        url =  self.url_page1
        soup = self.get_soup(url)
        return soup
    def get_soup(self, url):
        page = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(page)
        return soup
    def get_search_page_links(self, num_pages):
        article_list = []
        for i in range(1, num_pages + 1):
            temp_soup = self.get_soup(self.url_page1 + str('&page=') + str(i))
            for result in temp_soup.find_all('h1'):
                try:
                    if 'video' in result.a['href']:
                        continue
                    if 'http' in result.a['href']:
                        #print item.a['href']
                        article_list.append(result.a['href'])
                    else:
                        #print 'http://www.bloomberg.com/' + item.a['href']
                        article_list.append('http://www.bloomberg.com/' + result.a['href'])
                        print('http://www.bloomberg.com/' + result.a['href'])
                except:
                    continue
            #print 'Added page=' + str(i)
        return article_list
    
    def get_post_body(self, article_url):
        final_text = ""
        article_soup = self.get_soup(article_url)
        query = article_soup.find_all('div',  class_="article-body__content")
        for item in query:
            for text in item.find_all('p'):
                final_text = final_text + '\n\n' + str(text.text.encode('utf-8'))
        if final_text == "":
            return 0
        return final_text
    
    def get_post_date(self, article_url):
        final_text = ""
        article_soup = self.get_soup(article_url)
        result = article_soup.find('time', class_ = "published-at")
        try:
            return result['datetime']
        except:
            return None
    
    def get_post_author(self, article_url):
        final_text = ""
        article_soup = self.get_soup(article_url)
        result = article_soup.find('a', class_ = "author-link")
        try:
            return result.text.lstrip().rstrip()
        except:
            return None
    
    def get_post_title(self, article_url):
        final_text = ""
        article_soup = self.get_soup(article_url)        
        result = article_soup.find('title')
        try:
            return result.text.lstrip().rstrip()
        except:
            return None
    
    def make_info(self, pages = 1):
        final_df = pd.DataFrame()
        for url in self.get_search_page_links(pages):
            body = self.get_post_body(url)
            title = self.get_post_title(url)
            author = self.get_post_author(url)
            date = self.get_post_date(url)
            temp_series = pd.Series([title, author, date, body])
            final_df= final_df.append(temp_series, ignore_index = True)
        final_df.columns = ['title', 'author', 'date', 'text']
        return final_df

In [3]:
bloom_obj = BloombergSearch('ibm')

In [4]:
temp = bloom_obj.make_info(3)
temp.get_post_body

http://www.bloomberg.com/news/articles/2016-02-27/buffett-reiterates-praise-for-ibm-amex-leaders-amid-share-slump
http://www.bloomberg.com/news/articles/2016-02-25/ibm-changes-reporting-segments-in-a-move-toward-transparency
http://www.bloomberg.com/news/articles/2016-02-25/ims-health-pays-its-ceo-more-than-ibm-s-to-keep-him-from-leaving
http://www.bloomberg.com/news/articles/2016-02-29/sec-insider-trading-case-testing-new-standards-goes-to-jury
http://www.bloomberg.com/news/articles/2016-02-24/ibm-paid-1-3-billion-to-acquire-cleversafe-in-hybrid-cloud-push
http://www.bloomberg.com/news/articles/2016-02-28/buffett-hits-negative-drumbeat-clinton-wins-sc-saturday-wrap
http://www.bloomberg.com/news/articles/2016-02-26/buffett-s-empty-calendar-offers-billionaire-path-to-bigger-ideas
http://www.bloomberg.com/news/articles/2016-02-18/ibm-to-acquire-truven-health-analytics-for-2-6-billion-iksc5n0r
http://www.bloomberg.com/news/articles/2016-02-18/ibm-to-acquire-truven-health-analytics-for-2-6

/Users/OMalley/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


AttributeError: 'DataFrame' object has no attribute 'get_post_body'

In [175]:
class ForbesSearch:
    def __init__(self, search_term):
        self.search_term = search_term
        self.url_page1 = ('http://www.forbes.com/search/?q=' + str(self.search_term))

    def get_search_soup(self):
        url =  self.url_page1
        soup = self.get_soup(url)
        return soup
    def get_soup(self, url):
        page = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(page)
        return soup
    def get_search_page_links(self, num_pages):
        article_list = []
        for i in range(1, num_pages + 1):
            temp_soup = self.get_soup(self.url_page1 + str('&page=') + str(i))
            for result in temp_soup.find_all('h2'):
                try:
                    if 'video' in result.a['href']:
                        continue
                    if 'http' in result.a['href']:
                        #print item.a['href']
                        article_list.append(result.a['href'])
                    else:
                        #print 'http://www.bloomberg.com/' + item.a['href']
                        article_list.append('http://www.forbes.com/' + result.a['href'])
                except:
                    continue
            #print 'Added page=' + str(i)
        return article_list
    
    def get_post_body(self, article_url):
        final_text = ""
        article_soup = self.get_soup(article_url)
        query = article_soup.find_all('script')
        item = query[1]
       # final_text = str(item.text).encode('utf-8')
        indexA = item.text.find('"body":"<p>') 
        indexB = item.text.find('</p>","description"')
        
        pointer = indexA + 11
        while (pointer != indexB):
            if (item.text[pointer] == "<a>" or item.text[pointer] == "<"):
                while (item.text[pointer] != "</a>" and item.text[pointer] != ">"):
                    pointer = pointer + 1
            final_text = final_text + str((item.text[pointer]))
            pointer = pointer + 1
        final_text = final_text.replace('\\r\\n\\r\\n', '')
        final_text = final_text.replace('\\', '')
        return final_text
    
    def make_info(self, pages = 1):
        for url in self.get_search_page_links(pages):
            body = self.get_post_body(url)
        return 0

In [176]:
bloom_obj = ForbesSearch('ibm')


In [177]:
txt = bloom_obj.get_post_body('http://www.forbes.com/sites/jeffmcmahon/2016/02/24/aes-teslas-battery-competitor-hedges-its-bets-on-energy-storage/print/')
txt

/Users/OMalley/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


'[caption id="attachment_490899468" align="" width="960"]> Chief Executive Officer and President of The AES Corporation Andres Gluski . (Photo by Leigh Vogel/Getty Images for Concordia Summit)[/caption]Earlier this month The AES Corporation >announced> partnerships designed to sell&nbsp;its Advancion utility-scale battery system&nbsp;around the globe. But AES did not include income&nbsp;from those partnerships in the 2016&nbsp;outlook it released today.AES CEO Andres Gluski nonetheless expects the battery storage market to be "quite large" in five years&mdash;not quite as enthusiastic an assessment as Tesla Motors CEO Elon Musk\'s description of the market as ">staggeringly gigantic>.""In our current financial guidance, we do not include any material income from third-party sales of our Advancion project. We believe this is prudent, since our alliance-based business model is in its initial phases," Gluski said in a conference call with analysts this morning.Gluski reported lower earnin